In [68]:
# The goal of this notebook is to explore wildfire risk area
import geopandas as gpd
import fiona
import requests, zipfile, io

1. Définir la question à répondre

1.1 Combien de municipalités sont à risque de feu de forêt au Québec ?

1.2 Identifier les biens et les infrastructures les plus exposés aux feux de forêt à l’échelle des communautés.

2. Définir les sources de données pour cette analyse







In [69]:
# 2. Définir les sources de données pour cette analyse

# Municipalités
regionalUrl = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Regional/SDA_100k/SHP/BDAT(adm)_SHP.zip'
# Aéroport
airportsUrl = 'https://ws.mapserver.transports.gouv.qc.ca/swtq?service=wfs&version=2.0.0&request=getfeature&typename=ms:evenements&outfile=Aeroport&srsname=EPSG:4326&outputformat=geojson'
# villages autochtones
indigenousVillagesUrl = 'https://diffusion.mern.gouv.qc.ca/Diffusion/RGQ/Documentation/BDVA/Index_BDVA.zip'

firesUrl = 'https://diffusion.mffp.gouv.qc.ca/Diffusion/DonneeGratuite/Foret/PERTURBATIONS_NATURELLES/Feux_foret/FEUX_PROV_GPKG.zip'

# Empreintes de bâtiments
buildingFootPrint = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Local/Batiments/FGDB/QC_600019-54_Prevention_risques_sinistres_2019_fgdb.zip'

In [70]:
# Downloads the data sources, then format to DataFrame
responseBody = requests.get(airportsUrl, stream=True).json()
airports = gpd.GeoDataFrame.from_features(responseBody['features'], crs="EPSG:4326")
# airports.head(10)

In [79]:
# ref: https://pythonguides.com/download-zip-file-from-url-using-python/

import requests, zipfile, io

fireUrl = 'https://diffusion.mffp.gouv.qc.ca/Diffusion/DonneeGratuite/Foret/PERTURBATIONS_NATURELLES/Feux_foret/Feux_pt_ori_KMZ.zip'
filename = 'FEUX_CONT_SIMP_2021.kmz'

request = requests.get(fireUrl)

z = zipfile.ZipFile(io.BytesIO(r.content))

z.extractall("./dataSource")

In [71]:
# value = gpd.read_file('regional', url=regionalUrl)

import requests
import fiona
import geopandas as gpd

request = requests.get(firesUrl)
b = bytes(request.content)
with fiona.BytesCollection(b) as f:
    crs = f.crs
    gdf = gpd.GeoDataFrame.from_features(f, crs=crs)
    print(gdf.head())

r = requests.get(indigenousVillagesUrl, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
dataSource = z.extractall("./dataSource")

with fiona.BytesCollection(dataSource.read()) as src:
    crs = src.crs
    gdf = gpd.GeoDataFrame.from_features(src, crs=crs)
    print(gdf.head())